<a href="https://colab.research.google.com/github/YasminHeimann/Graph_Anomaly/blob/master/DGL_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic DGL tutorial, training a 2-layer GCN**



In [ ]:
pip install dgl


     |████████████████████████████████| 4.4 MB 4.2 MB/s 


In [ ]:
print("finished dgl installation")

finished dgl installation


In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


Get dataset from DGL for Node Classification

In [ ]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('\nNumber of categories:', dataset.num_classes)
# cora db has one graph
g = dataset[0]
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.

Number of categories: 7
Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False])}
Edge features
{}


**Define GCN model**

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions
print("features dim: ", g.ndata['feat'].shape[1])
# model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)


features dim:  1433


In [ ]:

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))


In [ ]:
#g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes) #.to('cuda')
train(g, model)
print("\nfinished training GCN")

In epoch 0, loss: 1.945, val acc: 0.108 (best 0.108), test acc: 0.127 (best 0.127)
In epoch 5, loss: 1.882, val acc: 0.540 (best 0.582), test acc: 0.553 (best 0.585)
In epoch 10, loss: 1.791, val acc: 0.546 (best 0.582), test acc: 0.564 (best 0.585)
In epoch 15, loss: 1.677, val acc: 0.558 (best 0.582), test acc: 0.569 (best 0.585)
In epoch 20, loss: 1.541, val acc: 0.652 (best 0.652), test acc: 0.643 (best 0.643)
In epoch 25, loss: 1.387, val acc: 0.696 (best 0.696), test acc: 0.697 (best 0.697)
In epoch 30, loss: 1.219, val acc: 0.718 (best 0.718), test acc: 0.723 (best 0.723)
In epoch 35, loss: 1.046, val acc: 0.724 (best 0.724), test acc: 0.731 (best 0.728)
In epoch 40, loss: 0.878, val acc: 0.744 (best 0.744), test acc: 0.738 (best 0.736)
In epoch 45, loss: 0.723, val acc: 0.754 (best 0.754), test acc: 0.738 (best 0.738)
In epoch 50, loss: 0.586, val acc: 0.760 (best 0.760), test acc: 0.738 (best 0.738)
In epoch 55, loss: 0.471, val acc: 0.756 (best 0.760), test acc: 0.742 (best 0

In [ ]:
print("hello gur")

hello gur
